In [13]:
import pandas as pd
import numpy as np

candidate_data = pd.read_csv('/home/keonwoo/anaconda3/envs/bgmRS/data/Audio library - 500.csv',encoding='utf-8')
candidate_data.drop(['Unnamed: 0','순서'],axis=1,inplace=True)
candidate_data = candidate_data.fillna(0)

mood_list = np.unique(candidate_data['메인무드'])
mood_list_ko = ['쿨한', '신나는', '우스운', '멋진', '평화로운', '로맨틱한', '무서운, 소름 끼치는, 미스터리한', '감동적인', '희망적인']
mood_dict = dict(zip(mood_list, mood_list_ko))

def change_label(x):
    return mood_dict[x]
    
candidate_data['메인무드'] = candidate_data['메인무드'].apply(lambda x: change_label(x))


from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import torch

# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")

model = AutoModelForSequenceClassification.from_pretrained("/home/keonwoo/anaconda3/envs/bgmRS/ckpt/eng/bigbird_0711_renew", num_labels=9)
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("/home/keonwoo/anaconda3/envs/bgmRS/ckpt/sbert_0706", num_labels=9)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [14]:
def recommend_mood(text):
    label_dict = {
        '희망적인' : 0,  
        '감동적인' : 1, 
        '무서운, 소름 끼치는, 미스터리한' : 2,
        '로맨틱한' : 3, 
        '평화로운' : 4,
        '멋진' : 5, 
        '우스운' : 6,
        '신나는' : 7, 
        '쿨한' : 8 
    }

    reversed_dict = dict(map(reversed, label_dict.items()))

    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    model.eval()
    with torch.no_grad():
        model_output = model(**encoded_input)

    output = torch.argmax(model_output[0][0])

    pred_mood = reversed_dict[int(output.numpy())]

    recommended_song_df = candidate_data[candidate_data['메인무드'] == pred_mood]

    print("적절한 테마를 고르시오: ", np.unique(recommended_song_df['테마']))

    return recommended_song_df

def recommend_song(recommended_song_df, tema, num_song=3):
    # tema = 'News / Current Affair'
    # num_song = 3

    final_recommended = recommended_song_df[recommended_song_df['테마'] == tema]

    if len(final_recommended) < num_song:
        print(final_recommended['곡제목(임시)'])
        RS = final_recommended['곡제목(임시)']
    else:
        print(final_recommended.sample(n=num_song)['곡제목(임시)'])
        RS = final_recommended.sample(n=num_song)['곡제목(임시)']

    return RS 

In [20]:
# 정확

text ="""
The Hwacheon Sancheoneo Ice Festival has been featured on CNN as one of the "7 Wonders of Winter." It attracts over 1 million visitors each year, continuing its reputation as one of the world's four major winter festivals. Featuring a wide variety of experiences and attractions, including mountain trout fishing on ice, a zip track stretching across the festival venue, and various snow and ice sculptures, the festival is one of Korea's top winter festivals, beloved by more and more tourists each year."""
    
text = text.replace('\n','')

df = recommend_mood(text)

print('\n')

songs = recommend_song(df, "Family / Kids").values

적절한 테마를 고르시오:  ['Advertising' 'Cafe' 'Comedy' 'Education' 'Family / Kids' 'Fashion'
 'Film / Drama / Dacumentary' 'History' 'Lifestyle / Leisure' 'Lounge'
 'News / Current Affair' 'SF' 'Sports' 'Time Lapse & Slomo' 'Video Game'
 'Vlog' 'sports/Leisure']


232    Jungle_house
25         Baby_dog
488        With_dad
Name: 곡제목(임시), dtype: object
